In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [2]:
from load_librairies import *
import xscale.signal.fitting as xsf

dir_hist_sst = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/tos/atl3/'
dir_hist_uas = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/uas/atl4/'

dir_rcp8_sst = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/tos/atl3/'
dir_rcp8_uas = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/uas/atl4/'


path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'

import warnings
warnings.filterwarnings('ignore')


lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1982
#period_end_1 = 1999

#period_str_2 = 2000
period_end_2 = 1999

def is_jja(month):
    return (month >= 6) & (month <= 8)
def is_mjj(month):
    return (month >= 5) & (month <= 7)

def nandetrend(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended_tmp= np.array(y) - m*x
    
    y_detrended  = xr.Dataset({'detrended': (['time'],y_detrended_tmp)}
                       ,coords={'time':y.time[:]})
    return y_detrended

# U wind

In [3]:
uwnd = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA_INTERIM/ERA_interim_uv_1979_2018_1x1.nc')

uwnd_12_tmp = uwnd.u10.sel(time = slice(datetime(period_str_1, 1, 1), datetime(period_end_2, 12, 31)))

uwnd_12_tmp = xr.concat([uwnd_12_tmp[:, :, 180:], uwnd_12_tmp[:, :, :180]], dim='longitude')
uwnd_12_tmp.coords['longitude'] = (uwnd_12_tmp.coords['longitude'] + 180) % 360 - 180

uwnd_12 = uwnd_12_tmp.chunk(chunks=None)
uwnd_12 = xsf.detrend(uwnd_12,dim='time',type='linear')

uwnd_12_ano,_ = Atools.ano_norm_t(uwnd_12.load())



In [4]:


uwnd_12_ano_Watl = uwnd_12_ano.where((uwnd_12_ano.longitude>=-40)&
                                                         (uwnd_12_ano.longitude<=-20)&
                                                         (uwnd_12_ano.latitude<=3)&
                                                         (uwnd_12_ano.latitude>=-3),
                                      drop=True).mean(dim='longitude').mean(dim='latitude')




# Z23

In [5]:

dir_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Data/'
data     = dir_data + 'ORA-S4/'


file_sst = 'global_SST_ORA_S4_1958_2017_montlhy.nc'
nc = xr.open_mfdataset(data+file_sst)
lon_oras = np.arange(-60,20,1)
lat_oras = np.arange(-30,30,1)
time_oras = nc.TIME
#sst_oras = nc.THETAO


dir_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Data/'
data = dir_data + 'ORA-S4/'
file = 'z23_depth_ORAS4_1958_2017_1x1.nc'
#nc   = Dataset(data+file)

nc= xr.open_mfdataset(data+file)
z23_tmp = nc.z23.sel(time = slice(datetime(period_str_1, 1, 1), datetime(period_end_2, 12, 31)))

z23tmp_2=np.ones((np.shape(z23_tmp)))*np.nan



for i in range(z23tmp_2.shape[1]):
    for j in range(z23tmp_2.shape[2]):
        if np.isfinite(z23_tmp[:,i,j]).all()==True:
            z23tmp_2[:,i,j] = sc.detrend(z23_tmp[:,i,j],type='linear')

X,Y = np.meshgrid(lon_oras,lat_oras) 
#
#
z23_12  = xr.Dataset({'z23': (['time','y','x'],z23tmp_2)}
                   ,coords={'time':np.array(z23_tmp.time),'lat':(['y','x'],np.array(Y)),'lon':(['y','x'],np.array(X))})



In [6]:
z23_12_ano,_ = Atools.ano_norm_t(z23_12.z23.load())



z23_12_ano_atl3 = z23_12_ano.where((z23_12_ano.lon>=-20)&
                                                         (z23_12_ano.lon<=0)&
                                                         (z23_12_ano.lat<=3)&
                                                         (z23_12_ano.lat>=-3),
                                   drop=True).mean(dim='y').mean(dim='x')

z23_12_ano_atl4 = z23_12_ano.where((z23_12_ano.lon>=-40)&
                                                         (z23_12_ano.lon<=-20)&
                                                         (z23_12_ano.lat<=3)&
                                                         (z23_12_ano.lat>=-3),
                                 drop=True).mean(dim='y').mean(dim='x')


In [7]:
thermo_slope_z23_12 = z23_12_ano_atl3 - z23_12_ano_atl4

# SST

In [16]:
data     = dir_data + 'ORA-S4/'
file_sst = 'global_SST_ORA_S4_1958_2017_montlhy.nc'
print('Loading the file :',data+file_sst)
nc = xr.open_mfdataset(data+file_sst)


SST_tmp= nc.THETAO[:,0,:,:]
SST_tmp =SST_tmp.sel(TIME = slice(datetime(1958, 1, 1), datetime(2017, 12, 31)))
tmp = xr.concat([SST_tmp[:,:,180:],SST_tmp[:,:,:180]],dim = 'LON') 
tmp.coords['LON'] = (tmp.coords['LON'] + 180)%360 - 180

lon = tmp.LON[:]
lat = tmp.LAT[:]
time_oras = tmp.TIME[:]
tmp = tmp.load()
sst_oras_tmp = tmp
 



sst_oras  = xr.Dataset({'sst': (['time','lat','lon'],sst_oras_tmp)}
                   ,coords={'time':np.array(time_oras),'lat':(np.array(lat)),'lon':(np.array(lon))})
sst_oras_12_tmp = sst_oras.sel(
    time=slice(datetime(period_str_1, 1, 1), datetime(period_end_2, 12, 31)))
sst_oras_12_test = sst_oras_12_tmp.sst.chunk(chunks=None)
sst_oras_12 = xsf.detrend(sst_oras_12_test,dim='time',type='linear')

sst_oras_12_ano, _ = Atools.ano_norm_t(sst_oras_12.load())

sst_oras_12_ano_atl3 = Atools.data_sub(sst_oras_12_ano,-20,0,-3,3).mean(dim='lon').mean(dim='lat')

Loading the file : /Users/aprigent/Documents/Thesis_GEOMAR/Data/ORA-S4/global_SST_ORA_S4_1958_2017_montlhy.nc


# take JJA anomalies

In [17]:
sst_12_ano_atl3_jja = sst_oras_12_ano_atl3.sel(time=is_jja(sst_oras_12_ano_atl3['time.month']))
uwnd_12_ano_Watl_jja =  uwnd_12_ano_Watl.sel(time=is_jja(uwnd_12_ano_Watl['time.month']))
thermo_slope_z23_12_jja = thermo_slope_z23_12.sel(time=is_jja(thermo_slope_z23_12['time.month']))
z23_12_ano_atl3_jja  = z23_12_ano_atl3.sel(time=is_jja(z23_12_ano_atl3['time.month']))

In [18]:
sst_12_ano_atl3_mjj = sst_oras_12_ano_atl3.sel(time=is_mjj(sst_oras_12_ano_atl3['time.month']))
z23_12_ano_atl3_mjj  = z23_12_ano_atl3.sel(time=is_mjj(z23_12_ano_atl3['time.month']))

# BF1

In [19]:


alpha_12_bf1,_, r_alpha_12_bf1, p_value_12_bf1, std_12_bf1    = stats.linregress(
                                                                       sst_12_ano_atl3_jja,
                                                                       uwnd_12_ano_Watl_jja) 

In [20]:


print('BF1 period 12:', np.round(alpha_12_bf1,2))

BF1 period 12: 0.59


# BF2

In [21]:
alpha_12_bf2_z23,_, r_alpha_12_bf2_z23, p_value_12_bf2_z23, std_12_bf2_z23    = stats.linregress(
                                                                       uwnd_12_ano_Watl_jja,
                                                                       thermo_slope_z23_12_jja) 
print('BF2 period 12 z23:', np.round(alpha_12_bf2_z23,2))

BF2 period 12 z23: 8.11


# BF3

In [22]:
alpha_12_bf3_jja_z23,_,r_alpha_12_bf3_jja_z23,_,std_12_bf3_jja_z23    = stats.linregress(z23_12_ano_atl3_jja,
                                                              sst_12_ano_atl3_jja) 

In [23]:
print('BF3 period 12 mjj z23:', np.round(alpha_12_bf3_jja_z23,3))

BF3 period 12 mjj z23: 0.132
